%md
Run `setup.ipynb` the first time - create fresh table - do not run setup in an automated job

In [0]:
from aeso_api import aeso
from datetime import datetime, timedelta
from pyspark.sql.functions import current_timestamp
import pytz

In [0]:
# AESO API key and instance initialization
aeso_api_key = 'b3b431c1c956490580df23f395786ba6'
aeso_instance = aeso(aeso_api_key)

# Get Current Date, and Current Date + 1
utc_time = pytz.utc.localize(datetime.utcnow())
calgary_time = utc_time.astimezone(pytz.timezone("America/Edmonton"))

start_date = (calgary_time - timedelta(hours=1)).strftime("%Y-%m-%d")
end_date = (calgary_time).strftime("%Y-%m-%d")

# Fetch pool price report from AESO API
resp = aeso_instance.get_pool_price_report(
    start_date=start_date, end_date=end_date
)

In [0]:
data = []
for r in resp:
    data.append(
        {
            "date": str(r.begin_datetime_utc),
            "poolPrice": str(r.pool_price)
        }
    )
df = spark.createDataFrame(data)
df.write.mode("append").saveAsTable("dhurley_catalog.benchmarks.aeso_pool_prices")